# IMPORT MODULES

In [ ]:
import pandas as pd
import numpy as np
import pandas_datareader.data as pdr
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import requests
import json

# PROCESS TRADING DATA

In [ ]:
response = requests.get("https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=demo")
alphadict = json.loads(response.text)
stock = pd.DataFrame(alphadict['Time Series (Daily)']).T
stock.index = pd.to_datetime(stock.index)
stock.columns = ['open', 'high', 'low', 'close', 'volume']
stock = stock.astype(float)
stock = stock.sort_index(ascending = True)
stock.head()

# GENERATE TRADING SIGNAL

In [ ]:
stock['SMA10'] = stock['close'].rolling(10).mean()
stock['SMA20'] = stock['close'].rolling(20).mean()
stock.dropna(inplace=True)
stock[['close','SMA10', 'SMA20']].plot(figsize=(20,5), grid = True, title='The 10 and 20 day simple moving averages of Microsoft'), plt.show();

# VECTORIZED BACKTESTING

In [ ]:
stock['position'] = np.where(stock['SMA10'] > stock['SMA20'], 1, -1)
stock['returns'] = np.log(stock['close']/stock['close'].shift(1))
stock['strategy'] = stock['position'].shift(1) * stock['returns']
stock.dropna(inplace=True)
stock.head()

# EVALUATE PERFORMANCE

In [ ]:
stock[['returns', 'strategy']].plot(figsize=(20,5), grid = True, title='Dual cross-over strategy versus long only MSFT returns'), plt.show();

In [ ]:
np.exp(stock[['returns', 'strategy']].sum())

In [ ]:
np.exp(stock['returns'])
stock[['returns', 'strategy']].describe()
